In [ ]:
!pip install tensorflow-addons
!pip install uszipcode
!pip install tzwhere
!pip install astral
!pip install pytorch-tabnet
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency, chisquare
from scipy.stats.contingency import expected_freq
from scipy.stats import kstest
from scipy.stats import nbinom
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import itertools
import tensorflow as tf
from tensorflow.keras import metrics
import tensorflow.keras as keras
import tensorflow_addons as tfa
from keras.optimizers import Adam
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from pytorch_tabnet.pretraining import TabNetPretrainer
from google.colab import drive
drive.mount('/content/drive')
TOM_FILES_DIR = "/content/drive/My Drive/Colab Notebooks/data science- transportation research/"
MAX_FILES_DIR = "/content/drive/My Drive/data science- transportation research/"
FILES_DIR = TOM_FILES_DIR

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.9 MB/s eta 0:00:00
  Created wheel for atomicwrites: filename=atomicwrites-1.4.1-py2.py3-none-any.whl size=6942 sha256=6577b74a16a3a55be15abd59675543f5ab5a5572506921000d2e6551921392ff
  Stored in directory: /root/.cache/pip/wheels/34/07/0b/33b15f68736109f72ea0bb2499521d87312b932620737447a2
Successfully built atomicwrites
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/drive


# create train test data

In [ ]:
FIRST_YEAR = 2013
SECOND_YEAR = 2022

In [ ]:
def filter_rows(chunk):
    return chunk[(chunk['YEAR'] == FIRST_YEAR) | (chunk['YEAR'] == SECOND_YEAR)]
chunk_size = 2000
filtered_chunks = []

for i, chunk in enumerate(pd.read_csv(f"{FILES_DIR}data_to_models.csv", chunksize=chunk_size)):
    filtered_chunk = filter_rows(chunk)
    filtered_chunks.append(filtered_chunk)
    if i%1000==0:
      print(f"iteration number {i}")

df = pd.concat(filtered_chunks)
df = df.drop(columns=["Unnamed: 0"])

iteration number 0
iteration number 1000
iteration number 2000
iteration number 3000
iteration number 4000
iteration number 5000
iteration number 6000
iteration number 7000
iteration number 8000


In [ ]:
sumed_df = df.groupby("ST_INDEX").sum()
index_of_only_zero = sumed_df["COLLISIONS"]!=0
df = df.set_index('ST_INDEX')
df = df[index_of_only_zero]
del sumed_df
del index_of_only_zero

<ipython-input-4-7756586a6da0>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[index_of_only_zero]


In [ ]:
def split_and_normalize(df):
  y = df.astype(np.float64)['COLLISIONS']
  X = df.astype(np.float64).drop('COLLISIONS', axis=1)


  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=4)
  for Xs in [X_train, X_test]:
    for col in Xs.columns:
        if np.all((Xs[col]==1) | (Xs[col]==0)):
            continue
        Xs[col] = (Xs[col] - Xs[col].mean())/Xs[col].std()
  return  X_train, X_test, y_train, y_test

# GLM no Regularization

In [ ]:
df_no_reg = df[(df["FerryRoute"]!=1)&(df["U-Turn"]!=1)]
df_no_reg = df_no_reg.drop(columns=["FerryRoute", "U-Turn"])
if 'ST_INDEX' in df_no_reg.columns:
  df_no_reg = df_no_reg.drop(columns=["ST_INDEX"])

df_no_reg["SMALL_STREET"] = ((df_no_reg["PathTrail"]==1) | (df_no_reg["Boardwalk"]==1) |
                      (df_no_reg["StepStreet"]==1) | (df_no_reg["Driveway"]==1) |
                      (df_no_reg["Alley"]==1)).astype(np.float64)
df_no_reg = df_no_reg.drop(columns=["PathTrail", "Boardwalk", "StepStreet", "Driveway", "Alley", "Street"])

In [ ]:
X_train, X_test, y_train, y_test = split_and_normalize(df_no_reg)
del df_no_reg

In [ ]:
model = sm.GLM(y_train, sm.tools.add_constant(X_train), family=sm.families.NegativeBinomial())
result = model.fit(method="IRLS")

# Print the model summary
print(result.summary())
y_pred = result.predict(sm.tools.add_constant(X_test))
mse_test = mean_squared_error(y_test, y_pred)/y_pred.shape[0]
print(mse_test)

with open('output_2013_2022_no_reg.txt', 'w') as file:
    file.write(str(result.summary()) + '\n')
    file.write(f"normalized MSE on test data is {mse_test}")

/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:             COLLISIONS   No. Observations:               840906
Model:                            GLM   Df Residuals:                   840867
Model Family:        NegativeBinomial   Df Model:                           38
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.8149e+05
Date:                Sat, 02 Sep 2023   Deviance:                   5.3044e+05
Time:                        06:41:38   Pearson chi2:                 1.05e+06
No. Iterations:                   100   Pseudo R-squ. (CS):            0.05887
Covariance Type:            nonrobust                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.46

# GLM with Regularization

In [ ]:
X_train, X_test, y_train, y_test = split_and_normalize(df)

In [ ]:
alpha = 0.005
l1_wt = 0.7

model = sm.GLM(y_train, sm.tools.add_constant(X_train), family=sm.families.NegativeBinomial())
result = model.fit_regularized(method='elastic_net', alpha=alpha, opt_method='IRLS', L1_wt= l1_wt)

y_pred = result.predict(sm.tools.add_constant(X_test))
mse_test = mean_squared_error(y_test, y_pred)/y_pred.shape[0]
print(mse_test)

with open('output_2013_2022.txt', 'w') as file:
    file.write(f"amount of features is {X_train.shape[1]}\n")
    file.write(str(np.round(result.params,3)) + '\n')
    file.write(f"normalized MSE on test data is {mse_test}")

/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


1.5526511138519733e-06


/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:1379: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not converge")


# GLM with 2 interaction variables

In [ ]:
important_features = ['SUMMONS', 'PEDESTRIAN_BINARY', 'INTERSECTIONS_BINARY', 'TURN_TRAFFIC_BINARY',
       'HUMPS', 'IS_SIGNED', 'SPEED_LIMIT']
important_df = df[important_features]
interaction_features = []

# Loop through all combinations of features and create interaction terms
for combo in itertools.combinations(important_df.columns, 2):
    interaction_feature = important_df[combo[0]] * important_df[combo[1]]
    interaction_features.append(interaction_feature)

interaction_df = pd.DataFrame(np.array(interaction_features)).T
interaction_column_names = [f"{combo[0]}_{combo[1]}" for combo in itertools.combinations(important_df.columns, 2)]
interaction_df.columns = interaction_column_names
final_df = pd.concat([df.reset_index(drop=True), interaction_df.reset_index(drop=True)], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = split_and_normalize(final_df)
del final_df

In [ ]:
alpha = 0.005
l1_wt = 0.7

model = sm.GLM(y_train, sm.tools.add_constant(X_train), family=sm.families.NegativeBinomial())
result = model.fit_regularized(method='elastic_net', alpha=alpha, opt_method='IRLS', L1_wt= l1_wt)

y_pred = result.predict(sm.tools.add_constant(X_test))
mse_test = mean_squared_error(y_test, y_pred)/y_pred.shape[0]
print(mse_test)

with open('output_2013_2022_inter.txt', 'w') as file:
    file.write(f"amount of features is {X_train.shape[1]}\n")
    for index, param in np.round(result.params, 3).iteritems():
        file.write(f'{index}: {param}\n')
    file.write(f"normalized MSE on test data is {mse_test}")

/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/usr/local/lib/python3.10/dist-packages/statsmodels/genmod/generalized_linear_model.py:1379: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not converge")


1.5523038841870067e-06


<ipython-input-8-60affffd59b4>:13: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, param in np.round(result.params, 3).iteritems():


# Asses models

In [ ]:
def calc_affect_mask(df_first_jan, df_second_jan, feature, div=10):
  if feature == "SUMMONS":
    return ((df_second_jan[feature].reset_index(drop=True)-df_first_jan[feature].reset_index(drop=True)))//div
  return (df_second_jan[feature].reset_index(drop=True)-df_first_jan[feature].reset_index(drop=True))

In [ ]:
def get_relevant_streets(df, affect_mask):
  affect_series = affect_mask.squeeze()
  return df[df.index.isin(affect_series[affect_series.astype(bool)].index)]

In [ ]:
def x_y_and_normalize(df):
  y = df.astype(np.float64)['COLLISIONS']
  X = df.astype(np.float64).drop('COLLISIONS', axis=1)
  for col in X.columns:
      if np.all((X[col]==1) | (X[col]==0)):
          continue
      X[col] = (X[col] - X[col].mean())/X[col].std()
  return  X, y

In [ ]:
def check_affect_of_features(df, model):
  df_first_jan = df[(df["YEAR"]==FIRST_YEAR)&(df["MONTH"]==1)]
  df_second_jan = df[(df["YEAR"]==SECOND_YEAR)&(df["MONTH"]==1)]
  df_first = df[(df["YEAR"]==FIRST_YEAR)]
  df_second = df[(df["YEAR"]==SECOND_YEAR)]
  print(f"In model {model}:")
  for feature in ["HUMPS", "PEDESTRIAN_BINARY", "TURN_TRAFFIC_BINARY", "SUMMONS", "INTERSECTIONS_BINARY"]:
    affect_mask = calc_affect_mask(df_first_jan, df_second_jan, feature)
    df_second_relevant_streets_exp = get_relevant_streets(df_second, affect_mask)
    df_first_relevant_streets = get_relevant_streets(df_first, affect_mask)
    df_second_relevant_streets_cont = df_second_relevant_streets_exp.copy()
    df_second_relevant_streets_cont[feature] = df_first_relevant_streets[feature]
    X_exp, y_exp = x_y_and_normalize(df_second_relevant_streets_exp)
    X_cont, y_cont = x_y_and_normalize(df_second_relevant_streets_cont)
    X_exp["YEAR"] = 1
    X_cont["YEAR"] = 1
    y_pred_exp = model.predict(X_exp.values)
    y_pred_cont = model.predict(X_cont.values)
    diff = y_pred_cont.mean() - y_pred_exp.mean()
    print(f"for feature {feature} the diff is {diff} and amount of {df_second_relevant_streets_exp.shape[0]}")

# MLP

In [ ]:
X_train, X_test, y_train, y_test = split_and_normalize(df)

In [ ]:
model = tf.keras.Sequential([
tf.keras.layers.Dense(1024, activation=tfa.activations.mish, input_shape=(X_train.shape[1],)),
keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(512, activation=tfa.activations.mish),
keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(256, activation=tfa.activations.mish),
keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(128, activation=tfa.activations.mish),
keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(64, activation=tfa.activations.mish),
keras.layers.BatchNormalization(),
tf.keras.layers.Dropout(0.5),
tf.keras.layers.Dense(32, activation=tfa.activations.mish),
keras.layers.BatchNormalization(),
tf.keras.layers.Dense(1)
])

batch_size = 65536

lr_schedule_cos = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=0.1,
    first_decay_steps=10*(X_train.shape[0]//batch_size),
    t_mul=2,
    m_mul=0.87,
    alpha=0.0,
    name=None
)


opt = keras.optimizers.Nadam(learning_rate=lr_schedule_cos)
model.compile(optimizer=opt, loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=batch_size, epochs=286, verbose=1)

# Make predictions on the test set and convert back to original scale
y_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)/y_pred.shape[0]
print(f"normalized MSE on test data is {mse_test}")


Epoch 1/286
13/13 [==============================] - 14s 86ms/step - loss: 1.1486
Epoch 2/286
13/13 [==============================] - 1s 84ms/step - loss: 0.4543
Epoch 3/286
13/13 [==============================] - 1s 85ms/step - loss: 0.4454
Epoch 4/286
13/13 [==============================] - 1s 85ms/step - loss: 0.4408
Epoch 5/286
13/13 [==============================] - 1s 85ms/step - loss: 0.4394
Epoch 6/286
13/13 [==============================] - 1s 86ms/step - loss: 0.4378
Epoch 7/286
13/13 [==============================] - 1s 85ms/step - loss: 0.4373
Epoch 8/286
13/13 [==============================] - 1s 84ms/step - loss: 0.4369
Epoch 9/286
13/13 [==============================] - 1s 85ms/step - loss: 0.4369
Epoch 10/286
13/13 [==============================] - 1s 84ms/step - loss: 0.4365
Epoch 11/286
13/13 [==============================] - 1s 85ms/step - loss: 0.4341
Epoch 12/286
13/13 [==============================] - 1s 85ms/step - loss: 0.4324
Epoch 13/286
13/13 [====

In [ ]:
check_affect_of_features(df, model)

In model <keras.engine.sequential.Sequential object at 0x7b366993de40>:
181/181 [==============================] - 0s 2ms/step
for feature HUMPS the diff is -2.0265579223632812e-06 and amount of 5784
602/602 [==============================] - 1s 2ms/step
for feature PEDESTRIAN_BINARY the diff is -0.0011313706636428833 and amount of 19236
77/77 [==============================] - 0s 2ms/step
for feature TURN_TRAFFIC_BINARY the diff is -0.00017273426055908203 and amount of 2460
135/135 [==============================] - 0s 2ms/step
for feature SUMMONS the diff is 0.006185397505760193 and amount of 4320
23/23 [==============================] - 0s 2ms/step
for feature INTERSECTIONS_BINARY the diff is 0.0 and amount of 720


In [ ]:
check_affect_of_features(df, model)

In model <keras.engine.sequential.Sequential object at 0x7dd9ad978400>:
181/181 [==============================] - 0s 2ms/step
for feature HUMPS the diff is 7.018446922302246e-06 and amount of 5784
602/602 [==============================] - 1s 2ms/step
for feature PEDESTRIAN_BINARY the diff is -0.0009408295154571533 and amount of 19236
77/77 [==============================] - 0s 2ms/step
for feature TURN_TRAFFIC_BINARY the diff is -0.0001939237117767334 and amount of 2460
29/29 [==============================] - 0s 2ms/step
for feature SUMMONS the diff is nan and amount of 912
23/23 [==============================] - 0s 2ms/step
for feature INTERSECTIONS_BINARY the diff is 0.0 and amount of 720


# TabNet

In [ ]:
X_train, X_test, y_train, y_test = split_and_normalize(df)

In [ ]:
unsupervised_model = TabNetPretrainer(
    n_d=4,
    n_a=4,
    n_steps=4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='entmax' # "sparsemax"
)

unsupervised_model.fit(
    X_train=X_train.values,
    eval_set=[X_test.values],
    pretraining_ratio=0.8,
    max_epochs=100,
    patience=15,
    batch_size=65536,
    virtual_batch_size=65536,
    num_workers=10,
)


tab_model = TabNetRegressor(n_d=4,
                            n_a=4,
                            n_steps=4)
tab_model.fit(
              X_train.values, y_train.values.reshape([-1, 1]),
              eval_set=[(X_test.values, y_test.values.reshape([-1, 1]))],
              max_epochs=100,
              patience=15,
              batch_size=65536,
              virtual_batch_size=65536,
              num_workers=10,
              from_unsupervised=unsupervised_model
              )

preds = tab_model.predict(X_test.values)

# Make predictions on the test set and convert back to original scale
y_pred = tab_model.predict(X_test.values)
mse_test = mean_squared_error(y_test.values.reshape([-1, 1]), y_pred)/y_pred.shape[0]
print(f"normalized MSE on test data is {mse_test}")


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 0  | loss: 1896.42103| val_0_unsup_loss_numpy: 11.352439880371094|  0:00:04s
epoch 1  | loss: 198.835 | val_0_unsup_loss_numpy: 4.244579792022705|  0:00:09s
epoch 2  | loss: 15.87779| val_0_unsup_loss_numpy: 4.837930202484131|  0:00:13s
epoch 3  | loss: 32.46551| val_0_unsup_loss_numpy: 3.4707698822021484|  0:00:17s
epoch 4  | loss: 3.62464 | val_0_unsup_loss_numpy: 2.034679889678955|  0:00:21s
epoch 5  | loss: 1.75935 | val_0_unsup_loss_numpy: 1.6458300352096558|  0:00:25s
epoch 6  | loss: 1.5609  | val_0_unsup_loss_numpy: 1.4325000047683716|  0:00:29s
epoch 7  | loss: 2.64764 | val_0_unsup_loss_numpy: 1.4136099815368652|  0:00:34s
epoch 8  | loss: 1.20354 | val_0_unsup_loss_numpy: 1.3925800323486328|  0:00:38s
epoch 9  | loss: 1.06269 | val_0_unsup_loss_numpy: 1.1501699686050415|  0:00:42s
epoch 10 | loss: 1.01405 | val_0_unsup_loss_numpy: 1.0362399816513062|  0:00:46s
epoch 11 | loss: 0.98426 | val_0_unsup_loss_numpy: 1.0236799716949463|  0:00:50s
epoch 12 | loss: 0.991   | va

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:118: UserWarning: Pretraining: mask_type changed from sparsemax to entmax
  warnings.warn(wrn_msg)
/usr/local/lib/python3.10/dist-

epoch 0  | loss: 0.78168 | val_0_mse: 0.5118  |  0:00:06s


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

epoch 98 | loss: 0.41686 | val_0_mse: 0.42487 |  0:10:18s


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 99 | loss: 0.4164  | val_0_mse: 0.42096 |  0:10:24s
Stop training because you reached max_epochs = 100 with best_epoch = 89 and best_val_0_mse = 0.41757


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


normalized MSE on test data is 1.4893684721513697e-06


In [ ]:
df_first_jan = df[(df["YEAR"]==FIRST_YEAR)&(df["MONTH"]==1)]
df_second_jan = df[(df["YEAR"]==SECOND_YEAR)&(df["MONTH"]==1)]
df_first = df[(df["YEAR"]==FIRST_YEAR)]
df_second = df[(df["YEAR"]==SECOND_YEAR)]
print(f"In model {tab_model}:")
for feature in ["INTERSECTIONS_BINARY", "HUMPS", "SUMMONS", "PEDESTRIAN_BINARY", "TURN_TRAFFIC_BINARY",]:
  affect_mask = calc_affect_mask(df_first_jan, df_second_jan, feature)
  df_second_relevant_streets_exp = get_relevant_streets(df_second, affect_mask)
  df_first_relevant_streets = get_relevant_streets(df_first, affect_mask)
  df_second_relevant_streets_cont = df_second_relevant_streets_exp.copy()
  df_second_relevant_streets_cont[feature] = df_first_relevant_streets[feature]
  X_exp, y_exp = x_y_and_normalize(df_second_relevant_streets_exp)
  X_cont, y_cont = x_y_and_normalize(df_second_relevant_streets_cont)
  X_exp["YEAR"] = 1
  X_cont["YEAR"] = 1
  y_pred_exp = tab_model.predict(X_exp.values)
  y_pred_cont = tab_model.predict(X_cont.values)
  diff = y_pred_cont.mean() - y_pred_exp.mean()
  print(f"for feature {feature} the diff is {diff} and amount of {df_second_relevant_streets_exp.shape[0]}")

In model TabNetRegressor(n_d=4, n_a=4, n_steps=4, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='entmax', input_dim=46, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]):
for feature INTERSECTIONS_BINARY the diff is 0.0 and amount of 720
for feature HUMPS the diff is -6.839632987976074e-06 and amount of 5784
for feature SUMMONS the diff is -0.003953337669372559 and amount of 1140
for feature PEDESTRIAN_BINARY the diff is -0.0009368211030960083 and amount of 19236
for feature TURN_TRAFFIC_BINARY the diff is -0.0012538284063339233 and amount of 2460


In [ ]:
check_affect_of_features(df, tab_model)

In model TabNetRegressor(n_d=4, n_a=4, n_steps=4, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=[], n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='entmax', input_dim=46, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1, grouped_features=[]):
for feature HUMPS the diff is -6.839632987976074e-06 and amount of 5784
for feature PEDESTRIAN_BINARY the diff is -0.0009368211030960083 and amount of 19236
for feature TURN_TRAFFIC_BINARY the diff is -0.0012538284063339233 and amount of 2460
for feature SUMMONS the diff is -8.392333984375e-05 and amount of 4320
for feature INTERSECTIONS_BINARY the diff is 0.0 and amount of 720


# MLP Grid Search

In [ ]:
def create_model(initial_learning_rate, t_mul, m_mul, alpha):
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation=tfa.activations.mish, input_shape=(X_train.shape[1],)),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(32, activation=tfa.activations.mish),
    keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
    ])

    batch_size = 131072

    lr_schedule_cos = tf.keras.optimizers.schedules.CosineDecayRestarts(
        initial_learning_rate=initial_learning_rate,
        first_decay_steps=10*(X_train.shape[0]//batch_size),
        t_mul=t_mul,
        m_mul=m_mul,
        alpha=alpha,
        name=None
    )

    opt = keras.optimizers.Nadam(learning_rate=lr_schedule_cos)
    model.compile(optimizer=opt, loss='mean_squared_error')
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'initial_learning_rate': [0.001, 0.01, 0.1],
    't_mul': [1.0, 2.0, 3.0],
    'm_mul': [0.87, 0.9, 0.95],
    'alpha': [0.0, 0.1, 0.2]
}

# Create the model
model = keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model, epochs=68, verbose=3)

# Create GridSearchCV instance
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')

# Fit the model to your data
grid_result = grid.fit(X_train, y_train)

# Print the best parameters found
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))